In [ ]:
import cv2,time
import torch
from numpy import random
import queue

from models.experimental import attempt_load
from utils.datasets import LoadStreams
from utils.general import check_img_size, non_max_suppression, scale_coords, set_logging, clean_str
from utils.plots import plot_one_box
from utils.torch_utils import select_device, time_synchronized

import sys
sys.path.append('/home/pi/project_demo/lib')
from McLumk_Wheel_Sports import *

In [ ]:
#复位舵机 Reset the servo
bot.Ctrl_Servo(1, 90)
bot.Ctrl_Servo(2, 25)

In [ ]:
#bgr8转jpeg格式 bgr8 to jpeg format
import enum


def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
#显示摄像头组件 Display camera components
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import time
# 线程功能操作库 Thread function operation library
import threading
import inspect
import ctypes

global classes
classes=classestemp=None
flag=rightrunflag=leftrunflag=1
leftflag=rightflag=pack1flag=pack2flag=stopflag=classflag=0
speed=5 # 速度不宜过快 The speed should not be too fast
image_widget = widgets.Image(format='jpeg', width=640, height=480)

##  线程关闭函数 Thread shutdown function

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)


# 定义图像识别线程 Define the image recognition thread

In [ ]:
def detect(weights='weights/best.pt', source='0', img_size=320, conf_thres=0.70, iou_thres=0.35, device=''):
    #Default: best.pt yolov5 model
    #best1.pt yolov5lite model
    global classes
    # Initialize
    set_logging()
    device = select_device(device)
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(img_size, s=stride)  # check img_size
    if half:
        model.half()  # to FP16

    # Set Dataloader
    dataset = LoadStreams(source, img_size=imgsz, stride=stride)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

    frame_count = 0
    start_time = time.time()

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Calculate FPS before processing the frame
        frame_count += 1
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time

        # Display FPS on the frame
        cv2.putText(im0s[0], f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Inference
        pred = model(img, augment=False)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s[i].shape).round()

                # Print results
                for *xyxy, conf, cls in reversed(det):
                    label = f'{names[int(cls)]} {conf:.2f}'
                    classes = f'{names[int(cls)]}'  
                    #classes = f'{names[int(cls)]}'

                    #print(classes)  # 打印识别的标签和置信度 Print the recognized labels and confidence
                    plot_one_box(xyxy, im0s[i], label=label, color=colors[int(cls)], line_thickness=3)

        # Stream results
        #cv2.imshow('Video0', im0s[0])
        global image_widget
        image_widget.value = bgr8_to_jpeg(im0s[0])

        # if cv2.waitKey(1) == ord('q'):  # Press 'q' to quit
        #     break

    #cv2.destroyAllWindows()

# 定义循迹控制线程 Define tracking control thread

In [ ]:
def tracking_control():
    global classes,speed,flag,leftflag,rightflag \
    ,pack1flag,pack2flag,stopflag,classestemp,rightrunflag,leftrunflag,classflag
    while True:  # 连续检测 Continuous detection
        # 从I2C读取巡线传感器数据 Read line sensor data from I2C
        track_data = bot.read_data_array(0x0a, 1)
        track = int(track_data[0])

        # 解析巡线传感器的状态 Analyze the status of the line patrol sensor
        x1 = (track >> 3) & 0x01  
        x2 = (track >> 2) & 0x01  
        x3 = (track >> 1) & 0x01  
        x4 = track & 0x01       
        """
        X2 X1 X3 X4
        |  |  |  |
        L1 L2 R1 R2
        """
        lineL1=x2
        lineL2=x1
        lineR1=x3
        lineR2=x4

        if classes != None:
            if(classes == 'stop'):
                classestemp=classes
                classflag=2
                flag=0
                if(lineL2 == 1 or lineR1 == 1):
                    move_backward(5)
                    time.sleep(0.1)
                    stop()
            elif(classes == 'run'):
                classestemp=classes
                classflag=0
                '''-----根据实际情况调整速度-----
                -Adjust speed according to actual situation-'''
                speed=5
                '''-----根据实际情况调整速度-----
                -Adjust speed according to actual situation-'''
                flag=1
                classes=None
        #flag=0
        if flag:
            #if (lineL1 == 0 and lineL2 == 0 and lineR1 == 0 and lineR2 == 0):
            if (lineL1 == 0 and lineL2 == 0 and lineR1 == 0 and lineR2 == 0) or (lineL1 == 0 and (lineR2 == 0 or lineR1 == 0)) or ( (lineL2 == 0 or lineL1 == 0) and lineR2 == 0):
            #if (lineL1 == 0 or lineR2 == 0):
                #根据识别结果调整行为 Adjust behavior based on recognition results
                if classes != None and classes !='run' and classes !='stop':
                    stop()
                    time.sleep(0.025)
                    bot.Ctrl_BEEP_Switch(1)
                    time.sleep(0.1)
                    bot.Ctrl_BEEP_Switch(0)
                    if classes=='right' :# 路标右转动作 Road sign right turn action
                        #print("right!!!!!!!!!!!!!")
                        bot.Ctrl_Servo(2, 5)
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                        move_forward(int(speed)) # 前进 go ahead
                        time.sleep(0.75)
                        rotate_right(int(speed*1.5))# 右转 Turn right
                        time.sleep(0.5)
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                        bot.Ctrl_Servo(2, 25)
                        rightrunflag=0
                        leftflag=0
                        classflag+=1
                    elif classes=='left' :# 路标左转动作 Road sign left turn action
                        #print("left!!!!!!!!!!!!!")
                        bot.Ctrl_Servo(2, 5)
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                        move_forward(int(speed))# 前进 go ahead
                        time.sleep(0.75)
                        rotate_left(int(speed*2))# 左转 Turn left
                        time.sleep(0.5)
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                        bot.Ctrl_Servo(2, 25)
                        leftrunflag=0
                        leftflag=1
                        classflag+=1
                    elif(classes == 'freeSpeed'):# 解除限速 Lift speed limit
                        speed=15#30
                    elif(classes == 'limitSpeed'):# 限制速度 Speed Limit
                        rightrunflag=1
                        leftrunflag=1
                        speed=5#20
                    elif(classes == 'beep'):# 鸣笛三声 Three blasts
                        rightrunflag=1
                        leftrunflag=1
                        bot.Ctrl_BEEP_Switch(1)
                        time.sleep(0.1)
                        bot.Ctrl_BEEP_Switch(0)
                        time.sleep(0.1)
                        bot.Ctrl_BEEP_Switch(1)
                        time.sleep(0.1)
                        bot.Ctrl_BEEP_Switch(0)
                        time.sleep(0.1)
                        bot.Ctrl_BEEP_Switch(1)
                        time.sleep(0.1)
                        bot.Ctrl_BEEP_Switch(0)
                        #print("beep")
                    elif(classes == 'one' and pack1flag ==0):# 倒入1库 Pour into 1 warehouse
                        bot.Ctrl_BEEP_Switch(1)
                        time.sleep(0.1)
                        bot.Ctrl_BEEP_Switch(0)
                        pack1flag=1
                        pack2flag=0
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                        move_backward(speed)# 后退 Back
                        time.sleep(0.3)
                        rotate_left(int(speed*4))# 左旋 Left Rotation
                        time.sleep(0.8)#0.8
                        move_backward(speed)# 后退 Back
                        time.sleep(0.4)
                        stop()# 停止
                        time.sleep(2)
                        move_forward(speed)# 前进 go ahead
                        time.sleep(0.75)
                        rotate_right(int(speed*4))# 右旋 Right Rotation
                        time.sleep(0.8)#0.8
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                    
                    elif(classes == 'two' and pack2flag ==0):# 倒入2库 Pour into 2 warehouses
                        bot.Ctrl_BEEP_Switch(1)
                        time.sleep(0.1)
                        bot.Ctrl_BEEP_Switch(0)
                        pack1flag=0
                        pack2flag=1
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                        move_backward(speed)# 后退 Back
                        time.sleep(0.9)
                        rotate_left(int(speed*4))# 左旋 Left Rotation
                        time.sleep(0.8)
                        move_backward(speed)# 后退 Back
                        time.sleep(0.4)
                        stop()# 停止
                        time.sleep(2)
                        move_forward(speed)# 前进 go ahead
                        time.sleep(0.75)
                        rotate_right(int(speed*4))# 右旋 Right Rotation
                        time.sleep(0.8)
                        '''-----根据实际情况调整速度，延时-----
                        -Adjust speed and delay according to actual conditions-'''
                classes=None
                stopflag=0
            if leftflag:# 左转优先 Left turn priority
                if(lineL1 == 0 and lineL2 == 0 and lineR1 == 0 and lineR2 == 0):
                    rightrunflag=1
                    leftrunflag=1
                    move_forward(int(speed))
                    stopflag=0
                elif lineL1 == 0 and (lineR2 == 0 or lineR1 == 0):  # 左锐角或左大弯 Left sharp angle or left sharp bend
                    #print("3")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_left(int(speed*1.5))  
                    time.sleep(0.15)
                elif( (lineL2 == 0 or lineL1 == 0) and lineR2 == 0):#右锐角：右大弯,0表示检测到黑线 Right acute angle: right big bend, 0 means black line is detected
                    #print("2")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_right(speed)
                    time.sleep(0.05)
                elif lineL1 == 0:  # 左最外侧检测 Left outermost detection
                    #print("4")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_left(speed)
                    time.sleep(0.02)
                elif lineR2 == 0:  # 右最外侧检测 Right outermost detection
                    #print("5")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_right((int(speed)))
                    time.sleep(0.05)
                elif lineL2 == 0 and lineR1 == 1:  # 中间黑线上的传感器微调车左转 The sensor on the middle black line fine-tunes the car to turn left
                    if leftrunflag==0:
                        classes=None
                    if rightrunflag==0:
                        classes=None
                    #print("6")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_left(int(speed))
                elif lineL2 == 1 and lineR1 == 0:  # 中间黑线上的传感器微调车右转 The sensor on the middle black line fine-tunes the car to turn right
                    if leftrunflag==0:
                        classes=None
                    if rightrunflag==0:
                        classes=None
                    #print("7")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_right(int(speed))
                elif lineL2 == 0 and lineR1 == 0:  # 都是黑色, 加速前进 All black, speed up
                    if(classflag)>1:
                        '''-----根据实际情况调整速度-----
                        -Adjust speed according to actual situation-'''
                        move_forward(5)
                        '''-----根据实际情况调整速度-----
                        -Adjust speed according to actual situation-'''
                    else:move_forward(speed)
            else:# 默认右转优先巡线 # Default right turn priority patrol
                if(lineL1 == 0 and lineL2 == 0 and lineR1 == 0 and lineR2 == 0):
                    rightrunflag=1
                    leftrunflag=1
                    move_forward(int(speed))
                    stopflag=0
                elif( (lineL2 == 0 or lineL1 == 0) and lineR2 == 0):#右锐角：右大弯,0表示检测到黑线 Right acute angle: right big bend, 0 means black line is detected
                    #print("2")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_right((int(speed*2)))
                    time.sleep(0.05)
                elif lineL1 == 0 and (lineR2 == 0 or lineR1 == 0):  # 左锐角或左大弯 Left sharp angle or left sharp bend
                    #print("3")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_left(int(speed*1.5))  
                    time.sleep(0.15)
                elif lineR2 == 0:  # 右最外侧检测 Right outermost detection
                    #print("5")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_right((int(speed)))
                    time.sleep(0.05)
                elif lineL1 == 0:  # 左最外侧检测 Left outermost detection
                    #print("4")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_left(int(speed*1.5))
                    time.sleep(0.05)
                elif lineL2 == 0 and lineR1 == 1:  # 中间黑线上的传感器微调车左转 The sensor on the middle black line fine-tunes the car to turn left
                    if leftrunflag==0:
                        classes=None
                    if rightrunflag==0:
                        classes=None
                    #print("6")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_left(int(speed))
                elif lineL2 == 1 and lineR1 == 0:  # 中间黑线上的传感器微调车右转 The sensor on the middle black line fine-tunes the car to turn right
                    if leftrunflag==0:
                        classes=None
                    if rightrunflag==0:
                        classes=None
                    #print("7")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    rotate_right(int(speed)) #右转
                elif lineL2 == 0 and lineR1 == 0:  # 都是黑色, 加速前进 All black, speed up
                    #print("8")
                    #print(lineL1,lineL2,lineR1,lineR2)
                    if(classflag)>1:
                        '''-----根据实际情况调整速度-----
                        -Adjust speed according to actual situation-'''
                        move_forward(5)
                        '''-----根据实际情况调整速度-----
                        -Adjust speed according to actual situation-'''
                    else:move_forward(speed)
            # 等待一段时间再进行下一次检测 Wait for a while before the next test
            #print("\n")
            time.sleep(0.01)
        elif flag==0:
            stop()

##  启动进程 Start the process

In [ ]:
display(image_widget)

thread1 = threading.Thread(target=detect)

thread1.daemon=True
thread1.start()

thread2 = threading.Thread(target=tracking_control)
thread2.daemon=True
thread2.start()

关闭进程 Close Process

In [ ]:
#如果小车未停止运动，请按复位键后再次执行 If the robot does not stop moving, press the reset button and execute again.
stop()
stop_thread(thread1)
stop_thread(thread2)